In [1]:
import sys
!{sys.executable} -m pip uninstall -y pyarrow
!{sys.executable} -m pip install pyarrow==15.0.0

Found existing installation: pyarrow 15.0.0
Uninstalling pyarrow-15.0.0:
  Successfully uninstalled pyarrow-15.0.0
  Using cached pyarrow-15.0.0-cp310-cp310-macosx_11_0_arm64.whl.metadata (3.0 kB)
Using cached pyarrow-15.0.0-cp310-cp310-macosx_11_0_arm64.whl (24.2 MB)


## 1. Load Energy Document Corpus
Use scraped or sample energy technical documents.

In [2]:
import pyarrow
import sys
print("pyarrow version:", pyarrow.__version__)
print("Python executable:", sys.executable)

pyarrow version: 15.0.0
Python executable: /Users/justin/miniconda3_Jun2023/envs/lora-llm/bin/python


In [3]:
with open('../data/energy_text.txt') as f:
    docs = [line.strip() for line in f if line.strip()]
docs[:2]

['Oil production forecasting is the prediction of future oil output from wells using historical and engineering data.',
 'Reservoir pressure is a key factor affecting oil production rates.']

## 2. Azure OpenAI Embeddings
Use text-embedding-ada-002 for vectorization.

In [4]:
import os
import openai
from typing import List

# Try to import sentence-transformers for local fallback
try:
    from sentence_transformers import SentenceTransformer
    _has_hf = True
except ImportError:
    _has_hf = False

# Check for Azure OpenAI credentials
AZURE_OPENAI_ENDPOINT = os.getenv('AZURE_OPENAI_ENDPOINT')
AZURE_OPENAI_KEY = os.getenv('AZURE_OPENAI_KEY')

def get_embedding(text: str) -> List[float]:
    """
    Returns embedding for text using Azure OpenAI if credentials are set, otherwise uses Hugging Face sentence-transformers.
    """
    if AZURE_OPENAI_ENDPOINT and AZURE_OPENAI_KEY:
        openai.api_type = 'azure'
        openai.api_base = AZURE_OPENAI_ENDPOINT
        openai.api_key = AZURE_OPENAI_KEY
        resp = openai.embeddings.create(input=[text], model='text-embedding-ada-002')
        return resp.data[0].embedding
    elif _has_hf:
        # Use Hugging Face all-MiniLM-L6-v2 as fallback
        if not hasattr(get_embedding, '_model'):
            get_embedding._model = SentenceTransformer('all-MiniLM-L6-v2')
        return get_embedding._model.encode(text).tolist()
    else:
        raise RuntimeError('No embedding provider available. Set Azure OpenAI credentials or install sentence-transformers.')


/Users/justin/miniconda3_Jun2023/envs/lora-llm/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


> **Tip:** This notebook will use Azure OpenAI for embeddings if credentials are set, otherwise it will fall back to Hugging Face sentence-transformers (all-MiniLM-L6-v2). To enable Azure, set `AZURE_OPENAI_ENDPOINT` and `AZURE_OPENAI_KEY` in your environment. To use the local fallback, ensure `sentence-transformers` is installed.

## 3. Store Embeddings in ChromaDB
Vector store for fast retrieval.

In [5]:
import chromadb
client = chromadb.Client()
collection = client.create_collection('energy_docs')
for i, doc in enumerate(docs):
    emb = get_embedding(doc)
    collection.add(documents=[doc], embeddings=[emb], ids=[str(i)])

## 4. RAG Pipeline: Retrieval + Generation
Retrieve relevant docs and generate answer with Azure OpenAI.

In [ ]:
def rag_query(query):
    q_emb = get_embedding(query)
    results = collection.query(query_embeddings=[q_emb], n_results=3)
    context = '\n'.join([d for d in results['documents'][0]])
    prompt = f'Context: {context}\n\nQuestion: {query}\nAnswer:'
    # For openai>=1.0.0
    response = openai.chat.completions.create(
        model='gpt-4',
        messages=[{'role': 'user', 'content': prompt}]
    )
    return response.choices[0].message.content

rag_query('How does reservoir pressure affect oil production?')

APIRemovedInV1: 

You tried to access openai.ChatCompletion, but this is no longer supported in openai>=1.0.0 - see the README at https://github.com/openai/openai-python for the API.

You can run `openai migrate` to automatically upgrade your codebase to use the 1.0.0 interface. 

Alternatively, you can pin your installation to the old version, e.g. `pip install openai==0.28`

A detailed migration guide is available here: https://github.com/openai/openai-python/discussions/742


## 5. Cost Tracking
Log token usage and estimate cost.

In [ ]:
# Example: log token usage
def log_cost(response):
    usage = response['usage']
    tokens = usage['total_tokens']
    cost = tokens * 0.00002  # Example cost per token
    print(f'Tokens: {tokens}, Estimated Cost: ${cost:.4f}')

## 6. Prompt Engineering
Try few-shot and chain-of-thought prompts.

In [ ]:
few_shot_prompt = '''
Context: Oil production depends on reservoir pressure.

        Question: What is reservoir pressure?
        Answer: Reservoir pressure is the pressure of fluids within a reservoir.
        
        Question: How does it affect oil production?
        Answer: Higher reservoir pressure generally increases oil production.
        
        Question: What happens when pressure drops?
        Answer: Oil production typically declines.
        '''

## 7. Evaluation
Evaluate retrieval accuracy and answer quality.